In [216]:
import random
import jieba.analyse
import pandas as pd

## 1. 加载停用词

In [217]:
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\百度停用词表.txt"
# stop_words_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\stopwords-master\\哈工大停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/哈工大停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_projects/resources/stopwords-master/中文停用词表.txt"

# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/中文停用词表.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/百度停用词表.txt"
stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/modified_stopwords.txt"
# stop_words_path = "/home/yang/PycharmProjects/NLP_project/resources/stopwords-master/四川大学机器智能实验室停用词库.txt"

stop_words = pd.read_csv(stop_words_path,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stop_words = stop_words['stopword'].values

## 2. 加载语料

In [218]:
# data_path = "C:\\Users\\Name\\PycharmProjects\\NLP_project\\resources\\面试-训练测试集.xlsx"
# data_path = "/home/yang/PycharmProjects/NLP_projects/resources/面试-训练测试集.xlsx"
data_path = "/home/yang/PycharmProjects/NLP_project/resources/面试-训练测试集.xlsx"
data = pd.read_excel(data_path)

# title_data = data['title']
# content_data = data['content']

## 3.定义文本预处理函数

* 去停用词，分词
* 参数 titles, contents, sentences, labels

In [219]:
def is_float(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
    return False

def is_percents(s):
    s = str(s)
    if s.count('.') ==1:
        left = s.split('.')[0]
        right = s.split('.')[1]
        right = right.split('%')[0]
        if right.isdigit():
            if left.count('-')==1 and left.startswith('-'):
                num = left.split['-'][-1]
                if num.isdigit():
                    return True
            elif left.isdigit():
                return True
            
    elif s.count(".") != 1:
        left = s.split('%')[0]
        if left.isdigit():
            return True
               
    return False

def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True


def preprocess_text(title, content, sentences, label):
    
#     处理标题
    title_info = jieba.lcut(title)

    title_info = [v for v in title_info if not str(v).isdigit()]#去数字
    title_info = list(filter(lambda x:x.strip(), title_info))   #去左右空格
    

    title_info = list(filter(lambda x:len(x)>1, title_info)) #长度为1的字符
    title_info = list(filter(lambda x:x not in stop_words, title_info)) #去掉停用词
    
#     print("标题信息是：" ,title_info)
#     sentences.append(segs)
#     sentences.append((" ".join(segs), label))# 打标签
            
#     处理文章内容        
    segs=jieba.lcut(content)
    segs = [v for v in segs if not str(v).isdigit()]#去数字
    segs = [v for v in segs if not is_float(v)] #去小数
    segs = [v for v in segs if not is_percents(v)] #去百分数

    segs = list(filter(lambda x:x.strip(), segs))   #去左右空格
    segs = list(filter(lambda x:len(x)>1, segs)) #长度为1的字符
    segs = list(filter(lambda x:x not in stop_words, segs)) #去掉停用词
    
    
#     print("内容信息是:", segs)
    segs = title_info + segs
    
    segs = [v for v in segs if is_all_chinese(str(v))] #去字母词
    
    
    sentences.append((" ".join(segs), label))# 打标签

    


## 4. 文本预处理

In [220]:
sentences = []

for index, row in data.iterrows():
#     print(row['title'], row['content'])
    title = row['title']
    content = row['content']
    label = row['label']
    preprocess_text(title, content, sentences, label)
    

In [221]:
# random.shuffle(sentences)
for sentence in sentences[:10]:
    print("-------")
    print("文章分词:", sentence[0])
    print("文章标签：", sentence[1])

-------
文章分词: 赵东 交易所 周线 放量 现在 反弹 不是 牛市 创始人 赵东 发微 博称 激动 交易所 周线 放量 现在 反弹 不是 牛市 牛会来 不是 现在 耐心
文章标签： 正面
-------
文章分词: 小幅 回调 美元 附近 宽幅 震荡 昨日 高涨 美元 目前 小幅 回调 主流 多数 小幅 回调 美元 跌幅 全球 数字 货币 市场 总价值 亿美元 成交量 亿美元 全球 均价 美元 三大 主流 交易所 现报 美元 现报 美元 币安现 美元 主流 数字 货币 表现 如下 暂报 美元 暂报 美元 暂报 美元 暂报 美元 暂报 美元 暂报 美元 小时 市值 排名 前百 币种 涨幅 前三为 跌幅 前三为 概念 板块 涨幅 前三为 技术 合约 工具 跨链 跌幅 前三为 人工智能 身份验证 联网 以上 涨跌幅 计算
文章标签： 正面
-------
文章分词: 短时 跌破 美元 刚刚 快速 下跌 短时 跌破 美元 最低 跌至 美元 略有 回升 现报 美元 今日 跌幅 提醒
文章标签： 负面
-------
文章分词: 过去 小时 推特 讨论 排名 第一 排名 三位 数据 显示 过去 小时 推特 讨论 排行 讨论 排名 第一 排名 第二位 排名 第三位 讨论 排名 四至 十位 分别
文章标签： 负面
-------
文章分词: 分析 美元 均线 支撑 有效 分析 未来 几天 可能 回落 美元 第一季度 引领 市场 走高 显示 疲软 迹象 中旬 月底 几乎 走势 相同 多头 空头 比率 以来 首次 跌至 以下 表明 看跌 情绪 日益加剧 价格 回调 看起来 可能 价格 借助 上升 移动 平均线 目前 美元 反弹 下跌 美元 情况 将会 减弱
文章标签： 负面
-------
文章分词: 分析 期货 多空 实力 反转 市场 情绪 逐渐 冷静 数据分析 显示 截止 点整 价格 收益 回报率 环比 昨日 跌幅 指数 处于 正常 区间 期货 方面 总多 单量 总空 单量 相较 此前 情况 市场 看好 情绪 已然 冷静 空头 势力 逐渐 攀升
文章标签： 负面
-------
文章分词: 过去 小时 推特 讨论 排名 第一 排名 三位 数据 显示 过去 小时 推特 讨论 排行 讨论 排名 第一 排名 第二位 排名 第三位 讨论 排名 四至 十位 分别
文章标签： 中性


## 5. 抽取词向量特征

In [222]:
# 1. 抽取特征， 定义词袋模型

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vec = CountVectorizer(
#     analyzer='word', # tokenise by character ngrams
#     ngram_range=(1,4),  # use ngrams of size 1 and 2
#     max_features=1000,  # keep the most common 1000 ngrams
# )

vec = TfidfVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,3),  # use ngrams of size 1 and 2
    max_features= None,  # keep the most common 1000 ngrams
    use_idf=1,smooth_idf=1,sublinear_tf=1
)



In [223]:
# 2. 语料切分

from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=1256)
# print(x_train)



## 6. 定义模型

In [224]:
# 3. 把训练数据转为词袋模型
vec.fit(x_train)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words=None, strip_accents=None,
                sublinear_tf=1, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=1, vocabulary=None)

In [225]:
# 4.算法建模与模型训练 （贝叶斯）
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
# 5. 评估结果 AUC值
print("多项贝叶斯模型评价，训练集正确率:",classifier.score(vec.transform(x_train), y_train))
print("多项贝叶斯模型评价，测试集正确率:",classifier.score(vec.transform(x_test), y_test))


多项贝叶斯模型评价，训练集正确率: 0.9054441260744985
多项贝叶斯模型评价，测试集正确率: 0.8533333333333334


In [226]:
# 4.算法建模与模型训练 （SVM）

from sklearn.svm import SVC
# svm = SVC(kernel='linear')
svm = SVC(C=1.0,kernel='linear',gamma='auto')
svm.fit(vec.transform(x_train), y_train)

print("SVM模型评价，训练集正确率",svm.score(vec.transform(x_train), y_train))
print("SVM模型评价，测试集正确率",svm.score(vec.transform(x_test), y_test))

SVM模型评价，训练集正确率 0.9684813753581661
SVM模型评价，测试集正确率 0.9133333333333333


## 目前准确率在测试集上为92%左右

In [227]:
import sklearn.neural_network as sk_nn
model = sk_nn.MLPClassifier(activation='tanh',solver='sgd',alpha=0.0001,learning_rate='adaptive',learning_rate_init=0.01,max_iter=1000)
model.fit(vec.transform(x_train),y_train)

print("神经网络模型评价，训练集正确率", model.score(vec.transform(x_train), y_train))

acc=model.score(vec.transform(x_test),y_test) #根据给定数据与标签返回正确率的均值
print('神经网络模型评价, 测试集正确率:',acc)

神经网络模型评价，训练集正确率 0.9856733524355301
神经网络模型评价, 测试集正确率: 0.92


In [233]:
from pyecharts.charts import Bar

bar = Bar()
bar.add_xaxis(["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"])
bar.add_yaxis("商家A", [5, 20, 36, 10, 75, 90])
# render 会生成本地 HTML 文件，默认会在当前目录生成 render.html 文件
# 也可以传入路径参数，如 bar.render("mycharts.html")
bar.render_notebook()

## 检验模型　https://blog.csdn.net/hao5335156/article/details/81268731